In [1]:
import torch
from transformers import AutoTokenizer
model_path = "./dienstag/chinese-roberta-wwm-ext"
classify_model = torch.load("./Model_saved/classify_model.pt")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
classify_model.to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def predict_single_sentence(sentence: str):
    # Tokenize the sentence with the same settings as during training
    inputs = tokenizer(sentence, return_tensors="pt", max_length=256, padding=True, truncation=True)
    # Move the input tensors to the correct device
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    
    # Set the model to evaluation mode (important if your model has layers like dropout or batchnorm)
    classify_model.eval()

    # Perform inference
    with torch.no_grad():
        outputs = classify_model(**inputs)
    #print(outputs)
    
    # Get the predicted class. This assumes that you're using a classification model
    # and that the model returns logits.
    # You might need to modify this depending on what your model's forward pass returns
    _, predicted = torch.max(outputs.logits, 1)
    
    return predicted.item()  # Convert the tensor to a Python scalar

/tmp/ipykernel_1743/3133227419.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classify_model = torch.load("./Model_saved/classify_model.pt")
/root/miniconda3/lib/python

In [2]:
import pandas as pd
df = pd.read_csv("../classificate_data/classificate_data.csv",encoding = 'utf-8')

In [3]:
n=0
id_list = []
for i in range(len(df['content'])):
    label = df['label'][i]
    id = predict_single_sentence(df['content'][i])
    id_list.append(id)
    if label == id:
        n = n+1
print(n/len(df['content']))

1.0


In [5]:
content = """
<结论>:符合
<符合性分析>:文件中明确规定了质量手册的编制及审批流程，包括编制部门、审核人、批准人以及发布的时机等细节。此外，还规定了质量手册的发放与保管方式，这些内容都体现了组织应编制质量手册的要求。
<原文>:1、质量手册的编制及审批
1.1 本手册由质量法规部组织人员编制，各部门确认后，经管理者代表审核、总经理批准后发布，并从批准之日起开始实施。
2、质量手册的发放与保管
2.1 本手册如需要发放，由管理者代表决定发放清单，质量法规部负责发放并作好发放记录。
"""
id = predict_single_sentence(content)
print(id)

1
